# NP-Hard Problems

The purpose of this assignment is to familiarize yourself with different approaches to solving NP-hard problems in practice, especially via integer programming.

As in previous assignments, we will use [NetworkX](https://networkx.github.io/) to manipulate graphs and [PuLP](http://pythonhosted.org/PuLP/) to solve linear and integer programs.

In [144]:
import networkx as nx
import pulp

We will use two graph datasets for this assignment, both available in [GML](https://en.wikipedia.org/wiki/Graph_Modelling_Language) format. The first is a well-known graph of the social network of a karate club [1]. The second is a network representing the western states power grid in the US [2].

In [145]:
karate = nx.read_gml('karate.gml')
power = nx.read_gml('power.gml')

[1] W. Zachary, An information flow model for conflict and fission in small groups, Journal of Anthropological Research 33, 452-473 (1977). 

[2] D. J. Watts and S. H. Strogatz, Collective dynamics of 'small world' networks, Nature 393, 440-442 (1998). 

## Independent Set

Recall that an *independent set* in a graph is a set of nodes such that no two nodes are adjacent (connected by an edge). Finding the maximum size of an independent set in a graph is an NP-hard problem, so there is no known polynomial-time algorithm to solve it.

### Problem 1

Provide an *integer program* formulation for this problem. This should look identical to a linear program formulation, except you will restrict the variables corresponding to the nodes to be integer instead of fractional.

The graph is denoted $G = (V, E)$. For this problem, you should use a binary variable $x_i \in \{0,1\}$ for each node $i \in N$, and you shouldn't need any other variables.

#### Solution:

TODO: formulate the integer program here


$ Objective: max \sum\limits_{i=1}^n x_i $

$s.t.\\
   x_i + x_j \leq 1 \hspace{1cm} (i,j) \in E \\
   x_i \in \{0,1\}  \hspace{1cm} i \in V \\ $


### Problem 2

Implement a function that solves the integer program given a graph as input.

In [146]:
def independent_set_ip(graph):
    """Computes the a maximum independent set of a graph using an integer program.
    
    Args:
      - graph (nx.Graph): an undirected graph
    
    Returns:
        (list[(int, int)]) The IP solution as a list of node-value pairs.
        
    """    
#Take a variable for every vertex of the graph, let it be 1 if vertex is occupied and 0 if it is not occupied. 
#We encode independence constraint by requiring that adjacent variables add up to 1
    #Creation of the problem
    prob = LpProblem("Independent set", LpMaximize)

    #Definition of the variables x_i
    # x_var = pulp.LpVariableDict
    Node_List = [str(i) for i in graph.nodes()]
    x_var = pulp.LpVariable.dict("y", Node_List, lowBound = 0,upBound = 1 ,  cat = pulp.LpInteger)

    #Contraints
    for i in graph.nodes():
        prob += x_var[str(i)] <= 1


    for edge in graph.edges():
        prob += x_var[str(edge[0])] + x_var[str(edge[1])] <= 1

    #Objective function
    prob += lpSum([x_var[str(i)] for i in graph.nodes()])
    status = prob.solve()
       
    res = []
    for i in graph.nodes():
        
        A = (int(i), int(x_var[str(i)].value()))
        res.append(A)
    
    
    return res

The following code outputs the size of the sets computed by your function.

In [147]:
def set_weight(solution):
    """Computes the total weight of the solution of an LP or IP for independent set.
    
    Args:
      - solution (list[int, float]): the LP or IP solution
    
    Returns:
      (float) Total weight of the solution
    
    """
    return sum(value for (node, value) in solution)

In [148]:
karate_ind_set = independent_set_ip(karate)
print "Size of karate set = ", set_weight(karate_ind_set)
power_ind_set = independent_set_ip(power)
print "Size of power set = ", set_weight(power_ind_set)

Size of karate set =  20
Size of power set =  2738


### Problem 3

Take the *linear programming relaxation* of your integer program and implement a function to solve it. This simply means that in your integer program, you should replace each constraint $x_i \in \{0,1\}$ with $0 \leq x_i \leq 1$.

In [149]:
def independent_set_lp(graph):
    """Computes the solution to the linear programming relaxation for the
    maximum independent set problem.
    
    Args:
      - graph (nx.Graph): an undirected graph
    
    Returns:
        (list[(int, float)]) The LP solution as a list of node-value pairs.
        
    """    
    #Creation of the problem
    prob = LpProblem("Independent set", LpMaximize)

    #Definition of the variables x_i
    
    Node_List = [str(i) for i in graph.nodes()]
    x_var = pulp.LpVariable.dict("y", Node_List, lowBound = 0   , upBound = 1  , cat='continuous')

    #Contraints
    for i in graph.nodes():
        prob += x_var[str(i)] <= 1


    for edge in graph.edges():
        prob += x_var[str(edge[0])] + x_var[str(edge[1])] <= 1

    #Objective function
    prob += lpSum([x_var[str(i)] for i in graph.nodes()])
    status = prob.solve()
    #print x_var[str(i)].value()
    res = []
    for i in graph.nodes():
        A = (int(i), float(x_var[str(i)].value()))
        res.append(A)
   
    return res

Let's see how the LP solutions compare to those of the IP.

In [150]:
karate_ind_set_relax = independent_set_lp(karate)
print "Value of karate set = ", set_weight(karate_ind_set_relax)
power_ind_set_relax = independent_set_lp(power)
print "Value of power set = ", set_weight(power_ind_set_relax)

Value of karate set =  20.5
Value of power set =  2758.0


A heuristic way to convert a fractional solution to an independent set is as follows. For each node $i$, include the node in the set if $x_i > 1/2$, and discard it otherwise. This will yield a set of $a$ nodes which have $b$ edges between them. By removing at most one node for each edge, this yields an independent set of size at least $a - b$.

Implement this rounding procedure.

In [151]:
def round_solution(solution, graph):
    """Finds the subgraph corresponding to the rounding of
    a solution to the independent set LP relaxation.
    
    Args:
      - solution (list[(int, float)]): LP solution
      - graph (nx.Graph): the original graph
      
    Returns:
        (nx.Graph) The subgraph corresponding to rounded solution
    
    """
    res = nx.Graph()
    for edge in graph.edges():
        res.add_edge(edge[0], edge[1])
    
    #Remove each node j if x_j <=  0.5
    for node in res.nodes():
        index = [k[0] for k in solution].index(node)
        if solution[index][1] <= 0.5:
            #Remove the node
            res.remove_node(node)
    return res

The following function assesses the quality of the heuristic approach.

In [152]:
def solution_quality(rounded, optimal):
    """Computes the percent optimality of the rounded solution.
    
    Args:
      - rounded (nx.Graph): the graph obtained from rounded LP solution
      - optimal: size of maximum independent set
    
    """
    num_nodes = rounded.number_of_nodes() - rounded.number_of_edges()
    return float(num_nodes) / optimal

Let's check the quality of this approach compared to the optimal IP solutions.

In [153]:
karate_rounded = round_solution(karate_ind_set_relax, karate)
karate_quality = solution_quality(karate_rounded, set_weight(karate_ind_set))
print "Quality of karate rounded solution = {:.0f}%".format(karate_quality*100)

power_rounded = round_solution(power_ind_set_relax, power)
power_quality = solution_quality(power_rounded, set_weight(power_ind_set))
print "Quality of power rounded solution = {:.0f}%".format(power_quality*100)

Quality of karate rounded solution = 90%
Quality of power rounded solution = 95%
